In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from utils import *

import gammy
from gammy.arraymapper import x
from gammy.models.bayespy import GAM

from scipy.interpolate import RegularGridInterpolator
from scipy import sparse
from scipy.optimize import minimize

In [2]:
X_train = pd.read_csv('data/X_train_PM2.5.csv')
X_test = pd.read_csv('data/X_test_PM2.5.csv')
y_train = pd.read_csv('data/y_train_PM2.5.csv')
y_test = pd.read_csv('data/y_test_PM2.5.csv')

In [3]:
from scipy.interpolate import RegularGridInterpolator
def custom_interp_matrix_1d(data, grid_points):
    x = data

    # Create an array representing the 'z' values at the grid points
    z = np.arange(len(grid_points))

    # Create a RegularGridInterpolator object to perform the interpolation
    interpolator = RegularGridInterpolator((grid_points,), z, method='linear', bounds_error=False, fill_value=None)

    # Calculate the interpolated values
    interp_values = interpolator(x)

    # Create the interpolation matrix
    A = np.zeros((len(x), len(grid_points)))

    for i, value in enumerate(interp_values):
        # Find the closest index in the z array
        closest_index = np.argmin(np.abs(z - value))
        A[i, closest_index] = 1

    return A

def smooth_diff1(input_col, xfit, mu=0, std=1e9, diff_mu=0, diff_std=1,
                 diff_order=2, sparse=True, name=None):
    nfit = len(xfit)
    D = bp.utils.diffmat(nfit, order=diff_order, sparse=sparse)
    diff_mu = diff_mu*np.ones(nfit-diff_order)
    diff_cov = diff_std**2*np.ones(nfit-diff_order)

    fmu = mu*np.ones(nfit)
    fcov = std**2*np.ones(nfit)

    return {
        'fun': lambda df: custom_interp_matrix_1d(df[input_col].values, xfit),
        'name': 'f({0})'.format(input_col) if name is None else name,
        'prior': {
            'B': np.vstack((D, np.eye(nfit))) if not sparse else
                bp.utils.vstack((D, bp.utils.eye(nfit))),
            'mu': np.concatenate((diff_mu, fmu)),
            'cov': np.concatenate((diff_cov, fcov))
        }
    }


In [4]:
## X_train first column to numpy array
X_train_array = X_train.to_numpy()

In [5]:
X_train = X_train.rename(columns={'0':'x1', '1':'x2', '2':'x3', '3':'x4', '4':'x5', '5':'x6', '6':'x7', '7':'x8', '8':'x9'})
data = pd.concat([X_train, y_train], axis=1)
data = data.rename(columns={'0':'y'})
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,y
0,-0.361490,-0.033399,-0.356207,0.631392,0.289869,-0.914473,0.041919,0.0,0.217221,-0.647179
1,0.948213,0.876160,0.909963,0.537599,0.738140,0.019989,0.447633,0.0,0.149394,1.026620
2,0.930334,0.878494,0.909963,0.548468,0.787355,0.585318,-0.161961,0.0,0.399047,0.952602
3,-0.010571,0.008107,1.026818,0.233113,0.107211,-0.414682,0.352415,0.0,0.000000,-0.010571
4,0.210482,-0.033399,-0.712414,-0.969208,0.289869,0.260448,-0.366530,0.0,0.149394,0.172240


In [6]:
X_test = X_test.rename(columns={'0':'x1', '1':'x2', '2':'x3', '3':'x4', '4':'x5', '5':'x6', '6':'x7', '7':'x8', '8':'x9'})
data_test = pd.concat([X_test, y_test], axis=1)
data_test = data_test.rename(columns={'0':'y'})
data_test.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,y
0,0.441933,0.233616,0.114673,-0.969208,-0.26186,0.705643,-1.081913,0.000000,1.087314,0.410755
1,-0.536356,-0.423692,0.249502,0.247847,0.00000,-0.442414,0.465470,0.000000,-0.494321,-0.711222
2,0.804819,0.677535,-0.504046,0.651086,0.63093,-0.575683,-0.125932,0.000000,-15.833196,0.807600
3,-1.204142,-1.136812,-0.712414,-0.874421,-0.63093,0.091770,0.891983,12.611541,0.992712,-1.379008
4,0.799216,0.751108,1.035455,0.651086,0.73814,0.048330,0.107734,0.000000,-2.952559,0.674733


In [7]:
x1_fit = np.linspace(-2, 1.6, 10)
x2_fit = np.linspace(-0.13, 1.7, 10)
x3_fit = np.linspace(-1.4, 1.8, 10)
x4_fit = np.linspace(-3.3, 1.7, 10)
x5_fit = np.linspace(-2, 2.3, 10)
x6_fit = np.linspace(-2.3, 1, 10)
x7_fit = np.linspace(-1.6, 1.9, 10)
x8_fit = np.linspace(0, 1, 10)
x9_fit = np.linspace(-5, 2, 10)

nfit1 = len(x2_fit)
nfit2 = len(x3_fit)
nfit3 = len(x4_fit)
nfit4 = len(x5_fit)
nfit5 = len(x6_fit)
nfit6 = len(x7_fit)
nfit7 = len(x8_fit)
nfit8 = len(x9_fit)

gam_specs = lambda logv: [
    smooth_diff1('x1', x1_fit, diff_std=np.exp(logv[0]), mu=np.mean(data.y.values), std=1),
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x2'].values, x2_fit),
        'name': 'f(x2)',
        'prior': {
            'B': bp.utils.diffmat(nfit1, order=2, periodic=True),
            'mu': np.zeros(nfit1),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit1-2), 0.1*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x3'].values, x3_fit),
        'name': 'f(x3)',
        'prior': {
            'B': bp.utils.diffmat(nfit2, order=2, periodic=True),
            'mu': np.zeros(nfit2),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit2-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x4'].values, x4_fit),
        'name': 'f(x4)',
        'prior': {
            'B': bp.utils.diffmat(nfit3, order=2, periodic=True),
            'mu': np.zeros(nfit3),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit3-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x5'].values, x5_fit),
        'name': 'f(x5)',
        'prior': {
            'B': bp.utils.diffmat(nfit4, order=2, periodic=True),
            'mu': np.zeros(nfit4),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit4-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x6'].values, x6_fit),
        'name': 'f(x6)',
        'prior': {
            'B': bp.utils.diffmat(nfit5, order=2, periodic=True),
            'mu': np.zeros(nfit5),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit5-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x7'].values, x7_fit),
        'name': 'f(x7)',
        'prior': {
            'B': bp.utils.diffmat(nfit6, order=2, periodic=True),
            'mu': np.zeros(nfit6),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit6-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x8'].values, x8_fit),
        'name': 'f(x8)',
        'prior': {
            'B': bp.utils.diffmat(nfit7, order=2, periodic=True),
            'mu': np.zeros(nfit7),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit7-2), 0.00001*np.ones(2))),
        }
    },
    {
        'fun': lambda df: custom_interp_matrix_1d(df['x9'].values, x9_fit),
        'name': 'f(x9)',
        'prior': {
            'B': bp.utils.diffmat(nfit8, order=2, periodic=True),
            'mu': np.zeros(nfit8),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit8-2), 0.00001*np.ones(2))),
        }
    }
]

In [8]:
gam_specs2 = lambda logv: [
    smooth_diff1('x1', x1_fit, diff_std=np.exp(logv[0]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x2', x2_fit, diff_std=np.exp(logv[1]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x3', x3_fit, diff_std=np.exp(logv[2]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x4', x4_fit, diff_std=np.exp(logv[3]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x5', x5_fit, diff_std=np.exp(logv[4]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x6', x6_fit, diff_std=np.exp(logv[5]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x7', x7_fit, diff_std=np.exp(logv[6]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x8', x8_fit, diff_std=np.exp(logv[7]), mu=np.mean(data.y.values), std=1),
    smooth_diff1('x9', x9_fit, diff_std=np.exp(logv[8]), mu=np.mean(data.y.values), std=1)
]


In [30]:
model = bp.models.GamModel('y', gam_specs2((np.log(0.001), np.log(0.01), np.log(0.01), np.log(0.01), np.log(0.1), np.log(0.1), np.log(0.1), np.log(0.1), np.log(0.1)))).fit(data, obs_cov=0.1**2)

In [31]:
y_train_pred = model.predict(data)
y_test_pred = model.predict(data_test)

# Performance
train_performance = {
    'Model': ['Linear Regression'],
    'Mean Square Error': [mean_squared_error(y_train, y_train_pred)],
    'Root Mean Square Error': [mean_squared_error(y_train, y_train_pred, squared=False)],
    'Mean Absolute Error': [mean_absolute_error(y_train, y_train_pred)],
    'R2 Score': [r2_score(y_train, y_train_pred)]
}

test_performance = {
    'Model': ['Linear Regression'],
    'Mean Square Error': [mean_squared_error(y_test, y_test_pred)],
    'Root Mean Square Error': [mean_squared_error(y_test, y_test_pred, squared=False)],
    'Mean Absolute Error': [mean_absolute_error(y_test, y_test_pred)],
    'R2 Score': [r2_score(y_test, y_test_pred)]
}

print(train_performance)
print(test_performance)


{'Model': ['Linear Regression'], 'Mean Square Error': [0.05088411528330935], 'Root Mean Square Error': [0.22557507682212888], 'Mean Absolute Error': [0.1562227509232043], 'R2 Score': [0.8858800568592028]}
{'Model': ['Linear Regression'], 'Mean Square Error': [0.052632942102091684], 'Root Mean Square Error': [0.22941870477816687], 'Mean Absolute Error': [0.1568881331246735], 'R2 Score': [0.88347059349213]}


In [9]:
like = lambda v: bp.models.GamModel('y', gam_specs2(v)).fit(data, obs_cov=0.5**2).L

In [13]:
v0=(np.log(8), np.log(5), np.log(5), np.log(5), np.log(5), np.log(5), np.log(5), np.log(5), np.log(5))
model_like = bp.models.GamModel('y', gam_specs2(minimize(like, v0, method='Nelder-Mead').x)).fit(data, obs_cov=0.5**2)

In [38]:
y_train_pred = model_like.predict(data)
y_test_pred = model_like.predict(data_test)


train_performance = {
    'Model': ['Linear Regression'],
    'Mean Square Error': [mean_squared_error(y_train, y_train_pred)],
    'Root Mean Square Error': [mean_squared_error(y_train, y_train_pred, squared=False)],
    'Mean Absolute Error': [mean_absolute_error(y_train, y_train_pred)],
    'R2 Score': [r2_score(y_train, y_train_pred)]
}

test_performance = {
    'Model': ['Linear Regression'],
    'Mean Square Error': [mean_squared_error(y_test, y_test_pred)],
    'Root Mean Square Error': [mean_squared_error(y_test, y_test_pred, squared=False)],
    'Mean Absolute Error': [mean_absolute_error(y_test, y_test_pred)],
    'R2 Score': [r2_score(y_test, y_test_pred)]
}

print(train_performance)
print(test_performance)



{'Model': ['Linear Regression'], 'Mean Square Error': [0.05166368482961839], 'Root Mean Square Error': [0.2272964690214487], 'Mean Absolute Error': [0.16044633957528354], 'R2 Score': [0.8841316834856312]}
{'Model': ['Linear Regression'], 'Mean Square Error': [0.053031370251350254], 'Root Mean Square Error': [0.23028541041792086], 'Mean Absolute Error': [0.16148745385342145], 'R2 Score': [0.8825884730193836]}
